Notebook written by [Zhedong Zheng](https://github.com/zhedongzheng)

![title](ctc.png)

In [1]:
"""
pip3 install librosa
pip3 install bs4
"""

from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
from tqdm import tqdm

import re
import os
import numpy as np
import tensorflow as tf
import librosa

In [7]:
PARAMS = {
    'num_epochs': 50,
    'batch_size': 30,
    'clip_norm': 5.0,
    'winstep': 0.01,
    'n_mfcc': 39,
}

In [3]:
def download():
    prefix = 'https://tspace.library.utoronto.ca'
    save_dir = './data/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    base_url = 'https://tspace.library.utoronto.ca/handle/1807/24'
    urls = [base_url+str(i) for i in range(488, 502)]

    for url in urls:
        soup = BeautifulSoup(urlopen(url).read(), 'html5lib')
        targets = soup.findAll('a', href=re.compile(r'/bitstream/.*.wav'))
        
        for a in tqdm(targets, total=len(targets), ncols=70):
            link = a['href']

            audio_save_loc = save_dir + link.split('/')[-1]
            if os.path.isfile(audio_save_loc):
                print("File Already Exists")
            urlretrieve(prefix+a['href'], audio_save_loc)

            with open(audio_save_loc.replace('.wav', '.txt'), 'w') as f:
                f.write('say the word ' + link.split('_')[-2])

In [4]:
def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return (indices, values, shape)


def train_input_fn(X, y):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(10000).batch(PARAMS['batch_size']).repeat(PARAMS['num_epochs'])
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()


def rnn_cell():
    return tf.nn.rnn_cell.GRUCell(PARAMS['n_mfcc'],
        kernel_initializer=tf.orthogonal_initializer())


def clip_grads(loss_op):
    variables = tf.trainable_variables()
    grads = tf.gradients(loss_op, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, PARAMS['clip_norm'])
    return zip(clipped_grads, variables)


def model_fn(features, labels, mode, params):
    seq_lens = tf.count_nonzero(tf.reduce_sum(features, -1), 1, dtype=tf.int32)
    
    outputs, _ = tf.nn.bidirectional_dynamic_rnn(
        rnn_cell(), rnn_cell(), features, seq_lens, dtype=tf.float32)
    outputs = tf.concat(outputs, -1)
    logits = tf.layers.dense(outputs, PARAMS['num_classes'])
    
    time_major = tf.transpose(logits, [1,0,2])
    decoded, log_prob = tf.nn.ctc_greedy_decoder(time_major, seq_lens)
    decoded = tf.to_int32(decoded[0])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        preds = tf.sparse_tensor_to_dense(decoded)
        return tf.estimator.EstimatorSpec(mode, predictions=preds)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        loss_op = tf.reduce_mean(tf.nn.ctc_loss(labels, time_major, seq_lens))
        edit_dist_op = tf.reduce_mean(tf.edit_distance(decoded, labels))

        lth = tf.train.LoggingTensorHook({'edit_dist': edit_dist_op}, every_n_iter=100)
        
        train_op = tf.train.AdamOptimizer().apply_gradients(
            clip_grads(loss_op), global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[lth])

In [5]:
download()

100%|███████████████████████████████| 200/200 [03:02<00:00,  1.09it/s]


In [8]:
wav_files = [f for f in os.listdir('./data') if f.endswith('.wav')]
text_files = [f for f in os.listdir('./data') if f.endswith('.txt')]

inputs, targets = [], []
for (wav_file, text_file) in tqdm(zip(wav_files, text_files), total=len(wav_files), ncols=70):
    path = './data/' + wav_file
    try:
        y, sr = librosa.load(path, sr=None)
    except:
        continue
    inputs.append(librosa.feature.mfcc(y = y,
                                       sr = sr,
                                       n_mfcc = PARAMS['n_mfcc'],
                                       hop_length = int(PARAMS['winstep']*sr)).T)
    with open('./data/'+text_file) as f:
        targets.append(f.read())

inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, dtype='float32', padding='post')

chars = list(set([c for target in targets for c in target]))
PARAMS['num_classes'] = len(chars) + 1

idx2char = {idx: char for idx, char in enumerate(chars)}
char2idx = {char: idx for idx, char in idx2char.items()}

targets = [[char2idx[c] for c in target] for target in targets]

inputs_val = np.expand_dims(inputs[-1], 0)
targets_val = targets[-1]

inputs_train = inputs[:-1]
targets_train = targets[:-1]
targets_train = tf.SparseTensor(*sparse_tuple_from(targets_train))

100%|█████████████████████████████| 2800/2800 [01:03<00:00, 44.17it/s]


In [9]:
estimator = tf.estimator.Estimator(model_fn)

estimator.train(lambda: train_input_fn(inputs_train, targets_train))

preds = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(inputs_val, shuffle=False)))

print('Prediction:', ''.join([idx2char[idx] for idx in preds[0]]))
print('Actual:', ''.join([idx2char[idx] for idx in targets_val]))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxg9xalhy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x119938b38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sx/

INFO:tensorflow:loss = 19.196615, step = 4501 (23.293 sec)
INFO:tensorflow:edit_dist = 0.32155505 (23.293 sec)
INFO:tensorflow:global_step/sec: 5.17865
INFO:tensorflow:loss = 20.466366, step = 4601 (19.310 sec)
INFO:tensorflow:edit_dist = 0.35303077 (19.310 sec)
INFO:tensorflow:Saving checkpoints for 4700 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxg9xalhy/model.ckpt.
INFO:tensorflow:Loss for final step: 16.368826.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxg9xalhy/model.ckpt-4700
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction: say the word 
Actual: say the word youth
